In [ ]:
import os
import xml.etree.ElementTree as et

import PIL.Image as Image
import pandas as pd
import requests
import tqdm

In [ ]:
image_aesthetics_path = '../../../DataSet/ImageAesthetics/beauty-icwsm15-dataset.tsv'
output_path = '../../DataSet/ImageAesthetics/'
api_key = '1188f7f86dd85503a7f28b983a3ca0c5'
proxies_port = 1087

In [ ]:
xml_path = os.path.join(output_path, 'xml_files')
if not os.path.exists(xml_path):
    os.mkdir(xml_path)

img_path = os.path.join(output_path, 'img_files')
if not os.path.exists(img_path):
    os.mkdir(img_path)

proxies = {'http': f'127.0.0.1:{proxies_port}', 'https': f'127.0.0.1:{proxies_port}'}

tsv_file = pd.read_csv(image_aesthetics_path, sep='\t', header=0)
processor = tqdm.tqdm(tsv_file['#flickr_photo_id'])
for photo_id in processor:
    if not os.path.exists(os.path.join(xml_path, f'{photo_id}.xml')):
        url = f'https://www.flickr.com/services/rest/?method=flickr.photos.getSizes&api_key={api_key}&photo_id={photo_id}&format=rest'
        xml_file = requests.get(url, proxies=proxies)
        open(os.path.join(xml_path, f'{photo_id}.xml'), 'wb').write(xml_file.content)

    if not os.path.exists(os.path.join(img_path, f'{photo_id}.jpg')):
        tree = et.parse(os.path.join(xml_path, f'{photo_id}.xml'))
        root = tree.getroot()
        if root.attrib['stat'] == 'ok':
            img_file = requests.get(root[0][3].attrib['source'], proxies=proxies)
            open(os.path.join(img_path, f'{photo_id}.jpg'), 'wb').write(img_file.content)
            Image.open(os.path.join(img_path, f'{photo_id}.jpg')).convert('RGB')
        elif root[0].attrib['code'] != '1':
            os.remove(os.path.join(img_path, f'{photo_id}.xml'))
            exit()
    else:
        Image.open(os.path.join(img_path, f'{photo_id}.jpg')).convert('RGB')

    processor.set_description(f'#ok={len(os.listdir(img_path))}; #fail={len(os.listdir(xml_path)) - len(os.listdir(img_path))}')